## Original Data

In [12]:
# Product Catalog data
product_catalog = {
    "CPU-4": { "Manufacturer": "Raspberry Pi", "Name": "Raspberry Pi 4 Computer", "Category": "Computer", "Price": 39.99 },
    "CPU-3B": { "Manufacturer": "Raspberry Pi", "Name": "Raspberry Pi 3B Computer", "Category": "Computer", "Price": 29.99 },
    "CASE-4": { "Manufacturer": "Flirc", "Name": "Case for Raspberry Pi 4", "Category": "Case", "Price": 15.95 },
    "CASE-3B": { "Manufacturer": "Flirc", "Name": "Case for Raspberry Pi 3B", "Category": "Case", "Price": 15.95 },
    "POWER-4": { "Manufacturer": "CanaKit", "Name": "5V 3.5A Power Adapter for Raspberry Pi 4", "Category": "Power", "Price": 12.99 },
    "POWER-3B": { "Manufacturer": "CanaKit", "Name": "5V 2.5A Power Adapter for Raspberry Pi 3B", "Category": "Power", "Price": 9.99 },
    "CARD-32": { "Manufacturer": "SanDisk", "Name": "32GB Micro SD Card", "Category": "Storage", "Price": 7.49 },
    "VIDEO-1": { "Manufacturer": "LCDBoard", "Name": "HDMI + VGA + DVI LCD Video Controller Board", "Category": "Video", "Price": 29.88 },
    "JOY-1": { "Manufacturer": "Sanwa", "Name": "1-Player Joystick and LED Button Kit", "Category": "Controls", "Price": 49.99 },
    "JOY-2": { "Manufacturer": "Sanwa", "Name": "2-Player Joystick and LED Button Kit", "Category": "Controls", "Price": 89.99 }
}

# State Tax Rates data
# state abbreviation: tax rate (0.08 = 8%)
state_tax_rates = {
    "CT": 0.06,
    "MA": 0.06,
    "NJ": 0.07,
    "NY": 0.08,
    "PA": 0.06
}

# Promo Codes data
# promo code: discount rate (0.08 = 8%)
promo_codes = {
    "VALENTINE": 0.08,
    "RETROCADE": 0.05,
    "PAC-MAN": 0.03
}

# Last Months Order data
last_months_order_data = (
    { "Order": 538, "State": "NY", "Parts": [ { "Part": "CPU-4", "Quantity": 1 } ], "Code": "" },
    { "Order": 539, "State": "NY", "Parts": [ { "Part": "CPU-4", "Quantity": 1 }, { "Part": "POWER-4", "Quantity": 1 } ], "Code": "" },
    { "Order": 540, "State": "CT", "Parts": [ { "Part": "CPU-3B", "Quantity": 1 }, { "Part": "VIDEO-1", "Quantity": 1 }, 
                                             { "Part": "JOY-1", "Quantity": 1 } ], "Code": "VALENTINE"},
    { "Order": 541, "State": "MA", "Parts": [ { "Part": "CARD-32", "Quantity": 1 } ], "Code": "" },
    { "Order": 542, "State": "NJ", "Parts": [ { "Part": "CPU-3B", "Quantity": 1 }, { "Part": "CASE-3B", "Quantity": 1 }, 
                                             { "Part": "POWER-3B", "Quantity": 1 }, { "Part": "VIDEO-1", "Quantity": 1 } ], 
     "Code": "PAC-MAN" },
    { "Order": 543, "State": "PA", "Parts": [ { "Part": "CPU-4", "Quantity": 1 }, { "Part": "CASE-4", "Quantity": 1 }, 
                                             { "Part": "POWER-4", "Quantity": 1 }, { "Part": "CARD-32", "Quantity": 1 }, 
                                             { "Part": "VIDEO-1", "Quantity": 1 }, { "Part": "JOY-2", "Quantity": 1 } ], 
     "Code": "RETROCADE" },
    { "Order": 544, "State": "NY", "Parts": [ { "Part": "CPU-3B", "Quantity": 1 }, { "Part": "CARD-32", "Quantity": 1 } ], 
     "Code": "" },
    { "Order": 545, "State": "NY", "Parts": [ { "Part": "CPU-3B", "Quantity": 1 }, { "Part": "CASE-3B", "Quantity": 1 }, 
                                             { "Part": "POWER-3B", "Quantity": 1 }, { "Part": "CARD-32", "Quantity": 1 }, 
                                             { "Part": "VIDEO-1", "Quantity": 1 }, { "Part": "JOY-2", "Quantity": 1 } ], 
     "Code": "PAC-MAN" },
    { "Order": 546, "State": "NY", "Parts": [ { "Part": "JOY-1", "Quantity": 1 } ], "Code": "VALENTINE" },
    { "Order": 547, "State": "MA", "Parts": [ { "Part": "CPU-3B", "Quantity": 2 }, { "Part": "CASE-3B", "Quantity": 2 } ], 
     "Code": "RETROCADE" },
    { "Order": 548, "State": "NJ", "Parts": [ { "Part": "CPU-4", "Quantity": 10 } ], "Code": "" }
)


# Data Analysis

## Create a Product Class

In [4]:
class Product:
    def __init__(self, part, manufacturer, name, category, price):
        self.part = part
        self.manufacturer = manufacturer
        self.name = name
        self.category = category
        self.price = price


## Create an Order Class

In [6]:
class Order:
    def __init__(self,order_number, tax_rate = 0, discount_rate = 0, products = None):
        self.order_number = order_number
        self.tax_rate = tax_rate
        self.discount_rate = discount_rate
        if products == None:
            self.products = []
        else:
            self.products = products
        self.getSubTotal()
        
    def addProduct(self, product):
        products = []
        self.products.append(product)
        
    def getSubTotal(self):
        order_sub_total = 0
        for product in self.products:
            order_sub_total = order_sub_total + product.price
        return order_sub_total
    
    def getTotal(self):
        order_total = self.getSubTotal() * (1 - self.discount_rate) * (1 + self.tax_rate)
        return order_total
    
    def getNumProducts(self):
        num_products = len(self.products)
        return num_products
        

## Code a createOrder Function
Create a function which converts order data into an order object (including its aggregate product objects).

In [8]:
def createOrder(order_number, parts_data, state_abbreviation, promo_code):
    global state_tax_rates, promo_codes, product_catalog
    tax_rate = state_tax_rates[state_abbreviation]
    discount_rate = promo_codes.get(promo_code, 0)
    order = Order(order_number, tax_rate, discount_rate)
    for part_data in parts_data:
        part = part_data['Part']
        quantity = part_data['Quantity']
        for i in range(quantity):
            product = Product(part, product_catalog[part]['Manufacturer'],product_catalog[part]['Name'],
                              product_catalog[part]['Category'],product_catalog[part]['Price'])
            order.addProduct(product)

    return order


## Load Orders From the Data

In [9]:
orders = []
global last_months_order_data
for data in last_months_order_data:
    order_object = createOrder(data["Order"],data["Parts"],data["State"],data["Code"])
    orders.append(order_object)


# Data Visualization

## Display an Order Summary Table

In [10]:
from beautifultable import BeautifulTable
table = BeautifulTable()

for order in orders:
    rows = []
    rows.append(order.order_number)
    rows.append(order.getNumProducts())
    rows.append("$ {:.2f}".format(order.getSubTotal()))
    rows.append("{:.1f} %".format(order.tax_rate*100))
    if order.discount_rate == 0:
        rows.append("-")
    else:
        rows.append("{:.1f} %".format(order.discount_rate*100))
    rows.append(order.getTotal())
    table.rows.append(rows)
table.columns.header = ["Order", "Size", "Sub-Total","Tax","Discount","Total"]
print(table)


+-------+------+-----------+-------+----------+---------+
| Order | Size | Sub-Total |  Tax  | Discount |  Total  |
+-------+------+-----------+-------+----------+---------+
|  538  |  1   |  $ 39.99  | 8.0 % |    -     | 43.189  |
+-------+------+-----------+-------+----------+---------+
|  539  |  2   |  $ 52.98  | 8.0 % |    -     | 57.218  |
+-------+------+-----------+-------+----------+---------+
|  540  |  3   | $ 109.86  | 6.0 % |  8.0 %   | 107.135 |
+-------+------+-----------+-------+----------+---------+
|  541  |  1   |  $ 7.49   | 6.0 % |    -     |  7.939  |
+-------+------+-----------+-------+----------+---------+
|  542  |  4   |  $ 85.81  | 7.0 % |  3.0 %   | 89.062  |
+-------+------+-----------+-------+----------+---------+
|  543  |  6   | $ 196.29  | 6.0 % |  5.0 %   | 197.664 |
+-------+------+-----------+-------+----------+---------+
|  544  |  2   |  $ 37.48  | 8.0 % |    -     | 40.478  |
+-------+------+-----------+-------+----------+---------+
|  545  |  6  

## Display a Monthly Revenue Summary Table

In [11]:
total_num_orders = len(orders)

total_revenue = 0
for order in orders:
    total_revenue = total_revenue + order.getTotal()

total_num_products = 0
for order in orders:
    total_num_products = total_num_products + order.getNumProducts()

average_order_size = total_num_products / total_num_orders

average_order_revenue = total_revenue / total_num_orders

from beautifultable import BeautifulTable
table = BeautifulTable()
table.rows.append(["total_num_orders",total_num_orders])
table.rows.append(["total_num_products",total_num_products])
table.rows.append(["total_revenue","$ {:.2f}".format(total_revenue)])
table.rows.append(["average_order_size", "{:.1f}".format(average_order_size)])
table.rows.append(["average_order_revenue","$ {:.2f}".format(average_order_revenue)])

for i in range(len(table.columns)):
    table.columns.alignment[i] = BeautifulTable.ALIGN_RIGHT

table.set_style(BeautifulTable.STYLE_BOX)
print(table)

┌───────────────────────┬───────────┐
│      total_num_orders │        11 │
├───────────────────────┼───────────┤
│    total_num_products │        40 │
├───────────────────────┼───────────┤
│         total_revenue │ $ 1304.79 │
├───────────────────────┼───────────┤
│    average_order_size │       3.6 │
├───────────────────────┼───────────┤
│ average_order_revenue │  $ 118.62 │
└───────────────────────┴───────────┘
